In [1]:
from nnunet_ext.network_architecture.nca.NCA2D import NCA2D
import torch
import torch.nn.functional as F
torch.set_grad_enabled(False)

In [2]:
device = "cuda"
x = torch.randn(1,1,128,128, device=device)

In [3]:
hidden_size0 = 32
nca0 = NCA2D(num_channels=16, num_input_channels=1, num_classes=0, hidden_size=hidden_size0, fire_rate=1, num_steps=10, use_norm=False)
nca0.to(device)

hidden_size2 = 10
nca1 = NCA2D(num_channels=5, num_input_channels=0, num_classes=0, hidden_size=hidden_size2, fire_rate=1, num_steps=10, use_norm=False)
nca1.fc0 = torch.nn.Conv2d(2 * (nca0.num_channels + nca1.num_channels), hidden_size2, kernel_size=1)
nca1.to(device);

In [4]:
def extended_nca_update(base_nca, new_nca, states):
    delta_state0 = base_nca.conv(states[0])
    delta_state1 = new_nca.conv(states[1])
    delta_state1 = torch.cat([states[0], states[1], delta_state0, delta_state1], dim=1)
    delta_state0 = torch.cat([states[0], delta_state0], dim=1)
    delta_state0 = base_nca.fc0(delta_state0)
    delta_state1 = new_nca.fc0(delta_state1)
    delta_state0 = base_nca.batch_norm(delta_state0)
    delta_state1 = new_nca.batch_norm(delta_state1)
    delta_state0 = F.relu(delta_state0, inplace=False)
    delta_state1 = F.relu(delta_state1, inplace=False)
    delta_state0 = base_nca.fc1(delta_state0)
    delta_state1 = new_nca.fc1(delta_state1)
    temp_state0 = states[0][:, base_nca.num_input_channels:] + delta_state0
    temp_state0 = torch.cat([states[0][:, :base_nca.num_input_channels], temp_state0], dim=1)
    return [temp_state0, states[1] + delta_state1]

In [5]:
state = nca0.make_state(x)
out0 = nca0.forward_internal(state)

In [6]:
state1 = torch.zeros(x.shape[0], nca0.num_channels - nca0.num_input_channels,
                    x.shape[2], x.shape[3], device=x.device)
state2 = torch.zeros(x.shape[0], nca1.num_channels, x.shape[2], x.shape[3], device=x.device)
states = [torch.cat([x, state1], dim=1), state2]

for _ in range(nca0.num_steps):
    states = extended_nca_update(nca0, nca1, states)

In [7]:
out0.shape, states[0].shape

(torch.Size([1, 16, 128, 128]), torch.Size([1, 16, 128, 128]))

In [8]:
(out0 == states[0]).all()

tensor(True, device='cuda:0')

In [9]:
states[1]

tensor([[[[ 0.7368,  0.1432,  0.4285,  ...,  0.5226,  0.2548, -0.1098],
          [ 0.1427, -0.2433,  1.4022,  ...,  0.2456,  1.1207, -0.1559],
          [-0.3884,  0.5746,  0.7781,  ...,  0.5638, -0.4462, -0.1741],
          ...,
          [ 0.5589,  1.2064,  0.2930,  ...,  1.1110,  0.8259,  0.1003],
          [ 1.3845, -0.5464,  0.9082,  ...,  0.5250,  0.5689, -0.0247],
          [-0.2142,  0.9061,  0.4481,  ..., -0.1562, -0.2074,  1.3203]],

         [[ 0.6733,  0.8128,  0.7441,  ...,  1.0092,  0.6164,  0.1534],
          [-0.0826,  0.8174,  0.9733,  ...,  0.4796,  0.8560,  0.1304],
          [-0.3310,  1.2854,  0.6273,  ...,  0.6898,  0.2545,  0.1888],
          ...,
          [ 0.6720,  0.5996,  0.3949,  ...,  0.5075,  0.6854, -0.0228],
          [ 0.7225, -0.2514,  0.6548,  ...,  0.2518,  0.2679,  0.4595],
          [-0.0163,  0.5124,  0.6474,  ...,  0.1029, -0.0240,  0.6378]],

         [[ 1.0296,  1.7044,  1.5744,  ...,  0.7552,  1.2714,  1.9306],
          [ 1.8797,  0.6165,  